# LangChain + vxdb

This notebook shows how to use **LangChain's embedding abstractions** with vxdb. LangChain gives you a unified interface to swap between providers (OpenAI, Hugging Face, Cohere, Ollama, etc.) without changing your vector store code.

**What this notebook covers:**
1. Using LangChain's `OpenAIEmbeddings` to generate vectors
2. Using LangChain's `HuggingFaceEmbeddings` for local, free embeddings
3. A reusable pattern for any LangChain embedding provider
4. Building a simple RAG (Retrieval-Augmented Generation) pipeline

**Prerequisites:**
```bash
pip install vxdb langchain-openai langchain-huggingface
```

> **Tip:** You only need the provider package you want. For example, `langchain-openai` for OpenAI, `langchain-huggingface` for local models.

In [ ]:
!pip install vxdb langchain-openai langchain-huggingface -q

## Option A: LangChain + OpenAI Embeddings

Requires `OPENAI_API_KEY` environment variable.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# LangChain handles API key from OPENAI_API_KEY env var automatically
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# LangChain provides two methods:
# - embed_documents(texts) → for indexing (batch)
# - embed_query(text)      → for searching (single)

test_vec = embeddings.embed_query("hello world")
print(f"OpenAI via LangChain: {len(test_vec)} dimensions")

## Option B: LangChain + Hugging Face (Local, Free)

No API key needed. Runs entirely on your machine.

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

test_vec = embeddings.embed_query("hello world")
EMBEDDING_DIM = len(test_vec)
print(f"HuggingFace via LangChain: {EMBEDDING_DIM} dimensions (local, free)")

## Step 2: Index documents with LangChain embeddings + vxdb

The pattern is always the same regardless of which LangChain embedding provider you chose above:
1. Call `embed_documents()` to batch-embed your texts
2. `upsert()` into a vxdb collection

In [ ]:
import vxdb

# Sample knowledge base
docs = [
    {"id": "k8s",     "text": "Kubernetes orchestrates containerized applications across clusters of machines, handling scaling, networking, and self-healing.",        "source": "infra"},
    {"id": "docker",  "text": "Docker packages applications into lightweight, portable containers that run consistently across development, staging, and production.",    "source": "infra"},
    {"id": "terraform","text": "Terraform is an infrastructure-as-code tool that lets you define cloud resources in declarative configuration files.",                    "source": "infra"},
    {"id": "react",   "text": "React is a JavaScript library for building user interfaces with a component-based architecture and virtual DOM for efficient rendering.",  "source": "frontend"},
    {"id": "fastapi", "text": "FastAPI is a modern Python web framework for building APIs, featuring automatic OpenAPI docs and async support out of the box.",          "source": "backend"},
    {"id": "postgres","text": "PostgreSQL is a powerful open-source relational database known for reliability, feature richness, and support for complex queries and JSON.","source": "database"},
    {"id": "redis",   "text": "Redis is an in-memory data store used as a cache, message broker, and real-time database, offering sub-millisecond latency.",              "source": "database"},
]

texts = [d["text"] for d in docs]

# Embed with LangChain (works with any provider)
vectors = embeddings.embed_documents(texts)

# Insert into vxdb
db = vxdb.Database()
collection = db.create_collection("devtools", dimension=EMBEDDING_DIM, metric="cosine")

collection.upsert(
    ids=[d["id"] for d in docs],
    vectors=vectors,
    metadata=[{"source": d["source"]} for d in docs],
    documents=texts,
)

print(f"Indexed {collection.count()} documents")

## Step 3: Search

In [ ]:
query = "How do I deploy containers to production?"

# embed_query for single queries (some providers optimize this differently)
query_vec = embeddings.embed_query(query)

# Vector search
print(f"Q: {query}\n")
print("Vector search:")
for r in collection.query(vector=query_vec, top_k=3):
    print(f"  → {r['id']:>12}  score={r['score']:.4f}  source={r['metadata']['source']}")

# Filtered: only infrastructure tools
print("\nFiltered (source=infra only):")
for r in collection.query(vector=query_vec, top_k=3, filter={"source": {"$eq": "infra"}}):
    print(f"  → {r['id']:>12}  score={r['score']:.4f}")

# Hybrid
print("\nHybrid search:")
for r in collection.hybrid_query(vector=query_vec, query=query, top_k=3, alpha=0.5):
    print(f"  → {r['id']:>12}  score={r['score']:.4f}")

## Step 4: Simple RAG pipeline

Retrieve relevant context from vxdb, then pass it to an LLM for answer generation.

> This cell requires `OPENAI_API_KEY` and uses `langchain-openai`. Swap in any LangChain LLM provider.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


def rag_query(question: str, top_k: int = 3) -> str:
    """Retrieve context from vxdb, then ask the LLM."""
    # 1. Embed the question
    q_vec = embeddings.embed_query(question)

    # 2. Retrieve relevant documents
    results = collection.query(vector=q_vec, top_k=top_k)

    # 3. Build context string from retrieved docs
    context_parts = []
    for r in results:
        original_text = next(d["text"] for d in docs if d["id"] == r["id"])
        context_parts.append(f"[{r['id']}] {original_text}")
    context = "\n".join(context_parts)

    # 4. Ask the LLM with the retrieved context
    prompt = f"""Answer the question based on the context below.

Context:
{context}

Question: {question}

Answer:"""

    response = llm.invoke(prompt)
    return response.content


# Try it
answer = rag_query("What tool should I use for infrastructure as code?")
print(answer)

## Swapping providers

The beauty of LangChain is that you can swap embedding providers with one line change:

```python
# OpenAI (API key required)
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Hugging Face (local, free)
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Cohere (API key required)
from langchain_cohere import CohereEmbeddings
embeddings = CohereEmbeddings(model="embed-english-v3.0")

# Ollama (local, free, needs ollama running)
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Google (API key required)
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
```

The rest of your vxdb code stays exactly the same.